### Importar Librerías 

In [26]:
%reset -f
import pandas as pd
import numpy as np
import os
from functools import partial, reduce
import operator

In [27]:
def getAllPaths(data_directory):
    #Input: directorio de los archivos.
    #Return: Arreglo con las rutas de los archivos.
    file_name = [os.path.join(data_directory,f) 
                 for f in os.listdir(data_directory)
                 if f.endswith(".csv")]
    return file_name

### Cargo todos los dataset de pruebas.

In [28]:
all_files = getAllPaths('data_sophia2/')
df = (pd.read_csv(f) for f in all_files)
df = pd.concat(df, ignore_index=True)

Tomar un sample de textos de noticias.

In [29]:
df = df.sample(10000)

Normalizacion

In [30]:
df = df['text'].dropna().str.lower().str.strip();

In [31]:
df

51570    la mañana de este miércoles, el ministro del i...
40898    17 personas fueron detenidas tras los serios i...
29381    cuando dc comics presentó a la nueva versión d...
14319    ni el cine ni la vida han sido lo mismo en año...
44026    el martes 8 de diciembre será feriado por una ...
                               ...                        
4061     el consejo para la transparencia se refirió al...
62723    tras referirse al desarrollo del consejo de ga...
41886    el pasado jueves se cumplió un año desde que f...
8207     la fiscalía metropolitana sur y la brigada de ...
35301    el ministro vocero de gobierno,  jaime belloli...
Name: text, Length: 9636, dtype: object

In [32]:
import string
import spacy
from spacy.lang.es.stop_words import STOP_WORDS
from spacy.lang.es import Spanish

spacy.prefer_gpu()
nlp = spacy.load("es_core_news_lg")

In [34]:
# Create our list of punctuation marks
punctuations = string.punctuation

# Create our list of stopwords
stop_words = spacy.lang.es.stop_words.STOP_WORDS
nlp.Defaults.stop_words |= {'.', ',', ':', ';', 'y', 'a', '?', '¿', '...', 'también', 'más', 'él', '%'}

# Creating our tokenizer function
def spacy_tokenizer(sentence):
    # Creating our token object, which is used to create documents with linguistic annotations.
    mytokens = nlp(sentence)

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    # return preprocessed list of tokens
    return mytokens

In [36]:
tokens = []

for doc in df.values:
    try:
        tokens.append(spacy_tokenizer(doc))
    except:
        continue

In [37]:
tokens[0][0]

'mañana'

In [38]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# Build the bigram and trigram models
bigram = gensim.models.Phrases(tokens, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[tokens], threshold=100)  

bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [39]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [40]:
# Remove Stop Words
#data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(tokens)

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [41]:
data_lemmatized[0]

['mañana',
 'miércoles',
 'ministro',
 'delgado',
 'llegar',
 'comuna',
 'reunir',
 'alcaldesa',
 'cathy',
 'barrigo',
 'conversar',
 'balacera',
 'dejar',
 'mujer',
 'fallecido',
 'persona',
 'grave',
 'lesión',
 'sentido',
 'secretario',
 'expresar',
 'gobierno',
 'repudiar',
 'ocurrir',
 'origen',
 'tipo',
 'delito',
 'requerir',
 'cambio',
 'estructural',
 'combatir',
 'banda',
 'hija',
 'mujer',
 'morir',
 'mamá',
 'recibir',
 'balazo',
 'cuello"leer',
 'delgado',
 'precisar',
 'tratar',
 'banda_criminal',
 'relacionado',
 'arma',
 'asociación_ilícito',
 'operar',
 'generar',
 'temor',
 'población',
 'ajustar',
 'terminar',
 'vida',
 'persona',
 'inocente',
 'ministro',
 'enfatizar',
 'agenda',
 'ejecutivo',
 'centrado',
 'llamar',
 'congreso',
 'seguir',
 'avanzar',
 'modificación',
 'ley',
 'arma',
 'banda_criminal',
 'evolucionar',
 'altura',
 'perseguir',
 'agregar',
 'delgado',
 'medida',
 'concreto',
 'maipú',
 'acordar',
 'aceleración',
 'construcción',
 'comisarío',
 'crea

In [62]:
id2word = corpora.Dictionary(data_lemmatized)
id2word[0]

'aceleración'

In [63]:
# assume the word 'b' is to be deleted, put its id in a variable
del_ids = [k for k,v in id2word.items() if v=='post_on' or v=='instagrama_post' or v=='año']

In [64]:
del_ids

[15, 1787, 1800]

In [65]:
# remove unwanted word ids from the dictionary in place
id2word.filter_tokens(bad_ids=del_ids)

In [66]:
texts = data_lemmatized

In [67]:
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('aceleración', 1),
  ('acordar', 1),
  ('afectar', 1),
  ('agenda', 1),
  ('agradecer', 1),
  ('agregar', 1),
  ('ajustar', 1),
  ('alcaldesa', 1),
  ('alcaldesa_maipú', 1),
  ('altura', 1),
  ('arma', 2),
  ('asegurar', 1),
  ('asociación_ilícito', 1),
  ('autoridad', 1),
  ('avanzar', 1),
  ('balacera', 1),
  ('balacera_feria', 1),
  ('balazo', 1),
  ('banda', 1),
  ('banda_criminal', 2),
  ('barrigo', 1),
  ('cambio', 1),
  ('carabinero', 1),
  ('cathy', 1),
  ('cathy_barriga', 1),
  ('centrado', 1),
  ('central', 1),
  ('combatir', 1),
  ('comisarío', 1),
  ('comuna', 2),
  ('concreto', 1),
  ('congreso', 1),
  ('construcción', 1),
  ('conversar', 1),
  ('creación', 1),
  ('cuello"leer', 1),
  ('decir', 1),
  ('dejar', 2),
  ('delgado', 4),
  ('delito', 1),
  ('dirigente', 1),
  ('disposición', 1),
  ('diálogo', 1),
  ('ejecutivo', 1),
  ('enfatizar', 1),
  ('espejoleer', 1),
  ('estructural', 1),
  ('evolucionar', 1),
  ('expresar', 1),
  ('fallecido', 1),
  ('generar', 1),
  (

In [68]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=9, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [69]:
print(lda_model.print_topics(num_words=5))

[(0, '0.016*"equipo" + 0.015*"partido" + 0.014*"jugar" + 0.010*"jugador" + 0.009*"ganar"'), (1, '0.016*"persona" + 0.008*"público" + 0.007*"entregar" + 0.006*"proceso" + 0.006*"medida"'), (2, '0.019*"carabinero" + 0.011*"investigación" + 0.010*"policía" + 0.010*"público" + 0.009*"caso"'), (3, '0.011*"vida" + 0.011*"mujer" + 0.009*"vivir" + 0.008*"dejar" + 0.008*"pasar"'), (4, '0.024*"caso" + 0.018*"país" + 0.013*"persona" + 0.011*"región" + 0.010*"contagio"'), (5, '0.014*"país" + 0.010*"unido" + 0.009*"internacional" + 0.009*"presidente" + 0.007*"empresa"'), (6, '0.011*"político" + 0.009*"presidente" + 0.009*"proyecto" + 0.008*"elección" + 0.007*"ministro"'), (7, '0.012*"serie" + 0.010*"favor_iniciar" + 0.008*"personaje" + 0.008*"película" + 0.008*"juego"'), (8, '0.037*"vehículo" + 0.020*"zona" + 0.015*"auto" + 0.014*"hora" + 0.011*"conductor"')]


In [70]:
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

C:\Users\razor\miniconda3\envs\nlp\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6     -0.178934  0.045411       1        1  21.390668
1     -0.127364 -0.077308       2        1  20.027761
4     -0.110957 -0.068599       3        1  14.358206
3     -0.047323  0.158125       4        1  11.264630
2     -0.008051 -0.237077       5        1   9.850629
5      0.007256 -0.144310       6        1   8.322265
0      0.012464  0.228951       7        1   6.764395
7      0.066511  0.131915       8        1   5.924489
8      0.386398 -0.037108       9        1   2.096957, topic_info=            Term         Freq        Total Category  logprob  loglift
301         caso  6246.000000  6246.000000  Default  30.0000  30.0000
447         país  6116.000000  6116.000000  Default  29.0000  29.0000
22    carabinero  2473.000000  2473.000000  Default  28.0000  28.0000
168      partido  2879.000000  2879.000000  Default  27.0000  27.0000
82       persona  8192.000000  8192.000000  Default  26.0000  26.0000
...          ...          ...          ...      ...      ...      ...
578    registrar   166.042081  1522.019189   Topic9  -5.0930   1.6491
1215      ciudad   143.108054  1101.421487   Topic9  -5.2416   1.8239
176     producir   139.833522  1183.799961   Topic9  -5.2648   1.7287
1263       noche   129.684545   673.323145   Topic9  -5.3401   2.2176
177     producto   125.580736  1030.467135   Topic9  -5.3723   1.7599

[533 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
5479      8  0.997101        _
594       1  0.058348  abogado
594       5  0.872068  abogado
594       6  0.067810  abogado
196       2  0.274145   acabar
...     ...       ...      ...
1031      3  0.226065     zona
1031      7  0.005666     zona
1031      9  0.311051     zona
5222      9  0.981703    ángel
3391      9  0.993181    ñuble

[1039 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 2, 5, 4, 3, 6, 1, 8, 9])

1: Politica  
2: Informacion?  
3: Salud/Covid  
4: Vida/Familia/Social?  
5: Crimen y Justicia  
6: Internacional?  
7: Deporte  
8: Entretenimiento/Arte/Cultura  
9: Zonas Geograficas/Crimen?  

In [73]:
def format_topics_documents(ldamodel=None, corpus=corpus, texts=texts):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_documents(ldamodel=lda_model, corpus=corpus, texts=texts)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
df_dominant_topic.head(10)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,2.0,0.2860,"carabinero, investigación, policía, público, c...","[mañana, miércoles, ministro, delgado, llegar,..."
1,1,2.0,0.6151,"carabinero, investigación, policía, público, c...","[persona, detener, serio, incidente, producir,..."
2,2,7.0,0.6374,"serie, favor_iniciar, personaje, película, jue...","[presentar, versión, justicia, debutar, evento..."
3,3,3.0,0.4799,"vida, mujer, vivir, dejar, pasar, seguir, habl...","[cine, año, pandemio, faltar, decir, vivir, re..."
4,4,4.0,0.4813,"caso, país, persona, región, contagio, salud, ...","[martes, feriar, festividad, religioso, capita..."
5,5,2.0,0.2966,"carabinero, investigación, policía, público, c...","[vía, mirada, crítico, marco, denunciar, inter..."
6,6,3.0,0.3581,"vida, mujer, vivir, dejar, pasar, seguir, habl...","[afectado, dramático, experiencia, mujer, hijo..."
7,7,6.0,0.5616,"político, presidente, proyecto, elección, mini...","[dejar, entrar, entrar, aforrar, quedar, inusi..."
8,8,2.0,0.3653,"carabinero, investigación, policía, público, c...","[hombre, año, permanecer, hospitalizado, riesg..."
9,9,1.0,0.4408,"persona, público, entregar, proceso, medida, p...","[reforma, promesa, campaño, acusación, vulnera..."
